# Deploy an AI Agent on AWS Inferentia2 with SageMaker

 This notebook demontrates how to deploy an open-ource LLM on a real-time AWS endpoint with Neuron devices using Hugging Face library `optimum-neuron`, `strands-agent` and AWS Sagemaker.



### 🎯 Goal

By the end of this notebook, you will know how to:

- **Deploy an LLM** optimized for instruction-following, reasoning, and tool-calling — specifically [`ibm-granite/granite-3.3-8b-instruct`](https://huggingface.co/ibm-granite/granite-3.3-8b-instruct).
- **Run it on an AWS Inferentia2 inference-optimized instance** (`inf2.24xlarge`) using a **SageMaker real-time endpoint**.
- **Use the deployed endpoint in an agentic setup**, including:
  - Calling external tools such as **web search**.
  - Connecting to and interacting with **MCP servers**, such as the Hugging Face MCP servers, to work with the Hugging Face Hub.

This notebook guides you from deployment to practical agent-enabled usage.

#### Configure logging and env

In [ ]:
%%writefile .env
HF_TOKEN=your_hf_token_here

In [56]:
from pathlib import Path
from dotenv import load_dotenv

# Load secrets from .env if present so the notebook picks up local configuration.
dotenv_path = Path.cwd() / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path)
    console.print(f"[green]Loaded environment from {dotenv_path}[/green]")
else:
    console.print(f"[yellow]No .env file found at {dotenv_path}; using existing environment variables.[/yellow]")


Loaded environment from /home/ec2-user/aws-llm-lab/neuron/.env

In [77]:
!pip install -q rich 

/home/ec2-user/.local/share/uv/python/cpython-3.13.2-linux-x86_64-gnu/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=2014688) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [30]:
import logging
from rich.console import Console
from rich.logging import RichHandler

console = Console()
logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(console=console, rich_tracebacks=True, markup=True)],
    force=True,
)
logger = logging.getLogger("neuron-agent")


### 1. Deploy an LLM on a SageMaker Inferentia2 real-time endpoint

***🔧 Configure SageMaker Session***

In [ ]:
!pip install sagemaker --upgrade --quiet

In [1]:
import sagemaker
import boto3

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


***🔑 Configure SageMaker Role***

In [43]:
sagemaker_role_name = "sagemaker-dlcs" # TODO: change to your own role name
iam = boto3.client("iam")
role = iam.get_role(RoleName=sagemaker_role_name)["Role"]["Arn"]

In [44]:
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::754289655784:role/sagemaker-dlcs
sagemaker session region: us-east-1


***📦 Prepare the model***

Before deploying the model, we need to **compile it for AWS Inferentia2**.

Inferentia and Trainium devices run models using the **Neuron runtime**, which requires models to be **compiled ahead of time** into an optimized Neuron executable.  
This compilation step transforms the original model (PyTorch or Transformers weights) into a hardware-optimized graph that can run efficiently on `inf2` instances.

Because we're working on a **Neuron-based instance**, compiling the model is a mandatory step before deployment.  

SageMaker will then use the **Optimum-Neuron vLLM container** to load the compiled artifacts and run them efficiently on Inferentia2.  
Instead of compiling the model at inference time, the container expects a **pre-compiled Neuron executable**, which it can serve directly through a real-time endpoint for fast and stable performance.

***🧩 Optimum Neuron Cache***

Hugging Face makes working with Neuron hardware much easier through the **Optimum Neuron Cache** ([`aws-neuron/optimum-neuron-cache`](https://huggingface.co/aws-neuron/optimum-neuron-cache)), a public repository that hosts **pre-compiled Neuron artifacts** for many popular models.  
Instead of spending time compiling models yourself—a process that can take minutes to hours—you can simply reuse these ready-made artifacts.

This means:

- **Instant startup** with no compilation step  
- **Consistent, reproducible performance** on Inferentia2 and Trainium  
- **Seamless integration** with the Optimum-Neuron vLLM containers, which can load these cached executables directly

If a matching compiled artifact is available, you can deploy your model on SageMaker immediately.

Learn more about the Optimum Neuron model cache [here](https://huggingface.co/docs/optimum-neuron/en/guides/cache_system).


***⚙️ Model compilation***

The model we selected, [`ibm-granite/granite-3.3-8b-instruct`]("https://huggingface.co/ibm-granite/granite-3.3-8b-instruct"), provides strong instruction-following, reasoning, and tool-calling abilities while remaining lightweight enough to run efficiently on a single Inferentia2 with a large input context necessary for agentic use-cases.

At the time of writing, **granite-3.3-8B-instruct** was not yet available as a pre-compiled Neuron artifact, so we compile it ourselves before deploying it on Inferentia2. We followed the steps from the official Optimum Neuron export tutorial (https://huggingface.co/docs/optimum-neuron/en/guides/export_model) to compile the model for Inferentia2.

To compile the model, we:

- Launched an **Inferentia2 EC2 instance** using the official **Hugging Face Neuron Deep Learning AMI**  
  (https://aws.amazon.com/marketplace/pp/prodview-gr3e6yiscria2).
- Connected to the instance via **SSH**.
- Used **Optimum Neuron** to compile the model with the following command:

  ```bash
  optimum-cli export neuron \
      --model ibm-granite/granite-3.3-8b-instruct \
      --sequence_length 16384 \
      --batch_size 1 \
      --num_cores 8 \
      --task text-generation \
      <folder-to-saved-compiled-artifacts>
- Pushed the compiled Neuron artifacts to the Hugging Face Hub:
[`florentgbelidji/granite-3.3-8b-instruct-neuron-bs-1-seq-16384-tp-8`](https://huggingface.co/florentgbelidji/granite-3.3-8b-instruct-neuron-bs-1-seq-16384-tp-8)

At the time of writing, AWS Inferentia2 does not support dynamic shapes for inference, which means that we need to specify our sequence length and batch size ahead of time. We chose these compilation parameters to match our agentic use case and the target hardware:

- **`--sequence_length 16384`**: Enables *large-context* inference, which is important when prompts include long tool definitions or multi-step reasoning typical in agent workflows.
- **`--batch_size 1`**: Prioritizes *low latency* over throughput, ideal for the demo where requests are sent sequentially rather than in batches.
- **`--num_cores 8`**: Maximizes hardware utilization on `inf2` instances.  
  An `inf2.24xlarge` exposes *6 Neuron devices = 12 Neuron cores* (learn more https://awsdocs-neuron.readthedocs-hosted.com/en/latest/about-neuron/arch/neuron-hardware/inf2-arch.html#aws-inf2-arch
  )

We set **`--num_cores 8`** because, at **deployment time**, the Neuron runtime requires that the model’s number of attention heads be divisible by the number of Neuron cores assigned to the model.  
Using 8 cores satisfies this constraint on Inferentia2 and ensures the model can be loaded and served correctly.




**🧱 Deployment Server: HuggingFace vLLM Inference Neuronx DLC**

For deployment, we use the **HuggingFace vLLM Inference Neuronx** Deep Learning Container from AWS ECR:

```
763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-vllm-inference-neuronx:0.11.0-optimum0.4.4-neuronx-py310-sdk2.26.1-ubuntu22.04
```

This container is built on [Optimum Neuron](https://huggingface.co/docs/optimum-neuron) and is designed to **serve Neuron-compiled models using vLLM on Inferentia2 instances**, combining Neuron runtime optimizations with vLLM's efficient inference mechanisms.

In [ ]:
llm_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-vllm-inference-neuronx:0.11.0-optimum0.4.4-neuronx-py310-sdk2.26.1-ubuntu22.04" 

***⚙️ Configure the SageMaker Endpoint with the Hugging Face SDK***

Now that we have a Neuron-compiled model and a deployment image, we can configure a SageMaker real-time endpoint using the **Hugging Face SageMaker SDK**

In [ ]:
import os 
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config
instance_type = "ml.inf2.24xlarge"
health_check_timeout = 3600  # additional time to load the model

config = {
    "SM_ON_MODEL": "florentgbelidji/granite-3.3-8b-instruct-neuron-bs-1-seq-16384-tp-8",
    "SM_ON_TENSOR_PARALLEL_SIZE": "8",# corresponds to the number of Neuron cores
    "SM_ON_BATCH_SIZE": "1",  # max batch size for the model
    "SM_ON_SEQUENCE_LENGTH": "16384",  # max length of generated text
    "SM_ON_ENABLE_AUTO_TOOL_CHOICE": "true", # enables tool choice
    "SM_ON_TOOL_CALL_PARSER": "granite", # enables tool call parsing with parser configured for granite
    "SM_ON_MAX_NUM_BATCHED_TOKENS": "8192", # max number of tokens in a batch, control prefill chunking
    "HF_TOKEN": os.environ["HF_TOKEN"],
}

endpoint_name = "granite-8b-instruct-neuron-bs-1-seq-16384-tp-8-demo-endpoint"

***Create Sagemaker Model***

In [ ]:

llm_model = HuggingFaceModel(
    role=role, 
    image_uri=llm_image, 
    env=config, 
    name=endpoint_name
    )

***🚀 Deploy***

In [10]:
llm_model._is_compiled_model = True

llm = llm_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    container_startup_health_check_timeout=health_check_timeout,
    inference_ami_version="al2-ami-sagemaker-inference-neuron-2",
)

-------------------!

Go to console.aws.amazon.com to check the status of your endpoint and the logs

In [ ]:
print(llm.endpoint_name)

***Reload already created endpoint***

If your endpoint is still running run the following:

In [ ]:
existing_endpoint_name = "granite-8b-instruct-neuron-bs-1-seq-163-2025-11-25-16-35-23-419" # TODO: change to your own endpoint name

In [ ]:
from sagemaker.predictor import Predictor
llm = Predictor(
    endpoint_name=existing_endpoint_name , 
    sagemaker_session=sess
                )

**Test**

In [71]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer

messages = [
    {"role": "user", "content": [{"type": "text", "text": "Hello my name is Huggy"}]},
]

parameters = {
    "max_tokens": 16,
    "temperature": 0.1,
    "stream": False,
}

llm.serializer = JSONSerializer()
llm.deserializer = JSONDeserializer()

payload = {"messages": messages, **parameters}
chat = llm.predict(payload)


In [74]:
chat

{'id': 'chatcmpl-d8a783e02fc34dee9b73a21e0427b984',
 'object': 'chat.completion',
 'created': 1764113456,
 'model': 'florentgbelidji/granite-3.3-8b-instruct-neuron-bs-1-seq-16384-tp-8',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hello Huggy, nice to meet you! How can I assist you today',
    'refusal': None,
    'annotations': None,
    'audio': None,
    'function_call': None,
    'tool_calls': [],
    'reasoning_content': None},
   'logprobs': None,
   'finish_reason': 'length',
   'stop_reason': None,
   'token_ids': None}],
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'prompt_tokens': 66,
  'total_tokens': 82,
  'completion_tokens': 16,
  'prompt_tokens_details': None},
 'prompt_logprobs': None,
 'prompt_token_ids': None,
 'kv_transfer_params': None}

In [76]:
print(chat["choices"][0]["message"]["content"])

Hello Huggy, nice to meet you! How can I assist you today


### 2. Create an Agent with Strands Agent


[Strands Agents](https://strandsagents.com/latest/) is an open-source Python SDK for building **production-grade AI agents** with minimal code.  
It provides a clean, model-driven framework for defining system prompts, tools, multi-step reasoning, and orchestration.

A key advantage for AWS users: **Strands natively supports SageMaker real-time endpoints**, allowing you to plug in your deployed LLMs directly as agent backends with no custom wrappers.

In [ ]:
!pip install -q 'strands-agents[sagemaker]' strands-agents-tools

In [80]:
from strands import Agent, tool
from strands.tools.mcp import MCPClient
from strands.models.sagemaker import SageMakerAIModel
from mcp.client.streamable_http import streamablehttp_client

***🔗 Link your endpoint to strands-agent***

In [103]:

model = SageMakerAIModel(
    endpoint_config={
        "endpoint_name": endpoint_name,
        "region_name": "us-east-1",  # or your region
    },
    payload_config={
        "max_tokens": 2048,
        "temperature": 0.2,
        "stream": False,
        
    }
)

***🕸️ Define a web-search tool for the agent***

To enable our agent to fetch fresh information from the web, we install the `duckduckgo-search` package, which provides a simple Python API for performing real-time DuckDuckGo queries.

In [111]:
!pip install -q ddgs

/home/ec2-user/.local/share/uv/python/cpython-3.13.2-linux-x86_64-gnu/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=2014688) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [115]:
from ddgs import DDGS

# define a tool to search the web
@tool()
def web_search(query:str, max_results=5)->dict:
    results = ""
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=max_results)]
    return results


***📞 My first call to the agent***

In [120]:

agent = Agent(model=model, tools=[web_search])
response = agent("What are AWS Neuron devices?")

[00:43:39] INFO     payload=<{                                                                     ]8;id=166264;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=813606;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#272\272]8;;\
                      "messages": [                                                                                
                        {                                                                                          
                          "role": "user",                                                                          
                          "content": [                                                                             
                            {                                                                                      
                              "text": "What are AWS Neuron devices?",                                              
                              "type": "text"                                                                       
                            }                                                                                      
                          ]                                                                                        
                        }                                                                                          
                      ],                                                                                           
                      "tools": [                                                                                   
                        {                                                                                          
                          "type": "function",                                                                      
                          "function": {                                                                            
                            "name": "web_search",                                                                  
                            "description": "web_search",                                                           
                            "parameters": {                                                                        
                              "properties": {                                                                      
                                "query": {                                                                         
                                  "description": "Parameter query",                                                
                                  "type": "string"                                                                 
                                },                                                                                 
                                "max_results": {                                                                   
                                  "default": 5,                                                                    
                                  "description": "Parameter max_results",                                          
                                  "type": "string"                                                                 
                                }                                                                                  
                              },                                                                                   
                              "required": [                                                                        
                                "query"                                                                            
                              ],                                      

[00:43:43] INFO     response=<{                                                                    ]8;id=815733;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=222146;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#438\438]8;;\
                      "id": "chatcmpl-129f0127990f45ba810ecf7ba419ba27",                                           
                      "object": "chat.completion",                                                                 
                      "created": 1764117820,                                                                       
                      "model":                                                                                     
                    "florentgbelidji/granite-3.3-8b-instruct-neuron-bs-1-seq-16384-tp-8",                          
                      "choices": [                                                                                 
                        {                                                                                          
                          "index": 0,                                                                              
                          "message": {                                                                             
                            "role": "assistant",                                                                   
                            "content": null,                                                                       
                            "refusal": null,                                                                       
                            "annotations": null,                                                                   
                            "audio": null,                                                                         
                            "function_call": null,                                                                 
                            "tool_calls": [                                                                        
                              {                                                                                    
                                "id": "chatcmpl-tool-49597574456f4d379c3d92192aaf3403",                            
                                "type": "function",                                                                
                                "function": {                                                                      
                                  "name": "web_search",                                                            
                                  "arguments": "{\"query\": \"AWS Neuron devices\",                                
                    \"max_results\": 5}"                                                                           
                                }                                                                                  
                              }                                                                                    
                            ],                                                                                     
                            "reasoning_content": null                                                              
                          },                                                                                       
                          "logprobs": null,                                                                        
                          "finish_reason": "tool_calls",                                                           
                          "stop_reason": null,                                                                     
                          "token_ids": null                           


Tool #1: web_search


           INFO     Error in engine duckduckgo: AttributeError("module 'httpcore._sync' has no          ]8;id=593331;file:///home/ec2-user/.venv/lib/python3.13/site-packages/ddgs/ddgs.py\ddgs.py]8;;\:]8;id=299665;file:///home/ec2-user/.venv/lib/python3.13/site-packages/ddgs/ddgs.py#199\199]8;;\
                    attribute 'http2'")                                                                            

[00:43:44] INFO     response:                                                                            ]8;id=782435;file://src/lib.rs\lib.rs]8;;\:]8;id=247367;file://src/lib.rs#464\464]8;;\
                    https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=AW           
                    S%20Neuron%20devices 200                                                                       

           INFO     response: https://search.brave.com/search?q=AWS+Neuron+devices&source=web 200        ]8;id=179040;file://src/lib.rs\lib.rs]8;;\:]8;id=685282;file://src/lib.rs#464\464]8;;\

[00:43:45] INFO     response:                                                                            ]8;id=588336;file://src/lib.rs\lib.rs]8;;\:]8;id=273967;file://src/lib.rs#464\464]8;;\
                    https://yandex.com/search/site/?text=AWS+Neuron+devices&web=1&searchid=4686631 200             

           INFO     payload=<{                                                                     ]8;id=393256;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=656068;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#272\272]8;;\
                      "messages": [                                                                                
                        {                                                                                          
                          "role": "user",                                                                          
                          "content": [                                                                             
                            {                                                                                      
                              "text": "What are AWS Neuron devices?",                                              
                              "type": "text"                                                                       
                            }                                                                                      
                          ]                                                                                        
                        },                                                                                         
                        {                                                                                          
                          "role": "assistant",                                                                     
                          "tool_calls": [                                                                          
                            {                                                                                      
                              "function": {                                                                        
                                "arguments": "{\"query\": \"AWS Neuron devices\", \"max_results\":                 
                    5}",                                                                                           
                                "name": "web_search"                                                               
                              },                                                                                   
                              "id": "chatcmpl-tool-49597574456f4d379c3d92192aaf3403",                              
                              "type": "function"                                                                   
                            }                                                                                      
                          ]                                                                                        
                        },                                                                                         
                        {                                                                                          
                          "role": "tool",                                                                          
                          "tool_call_id": "chatcmpl-tool-49597574456f4d379c3d92192aaf3403",                        
                          "content": "[{'title': 'Amazon EC2 AI Chips Architecture \u2014 AWS                      
                    Neuron Documentation', 'href':                                                                 
                    'https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/arch/neuron-h                 
                    ardware/neuron-devices.html', 'body': 'Amazon EC2 AI Chips ( Neuron Devices )                  
                    are the accelerated machine learning chips (e.g. I

[00:43:52] INFO     response=<{                                                                    ]8;id=37293;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=343691;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#438\438]8;;\
                      "id": "chatcmpl-57a9e3be4c3e47669965befe723bc5e5",                                           
                      "object": "chat.completion",                                                                 
                      "created": 1764117825,                                                                       
                      "model":                                                                                     
                    "florentgbelidji/granite-3.3-8b-instruct-neuron-bs-1-seq-16384-tp-8",                          
                      "choices": [                                                                                 
                        {                                                                                          
                          "index": 0,                                                                              
                          "message": {                                                                             
                            "role": "assistant",                                                                   
                            "content": "AWS Neuron devices refer to the accelerated machine                        
                    learning chips, specifically AWS Inferentia and AWS Trainium, which are                        
                    designed to enhance AI and deep learning workloads on Amazon EC2 instances.                    
                    These chips are optimized for high-density, scalable, and cost-effective                       
                    inference and training respectively. \n\nAWS Neuron is a software development                  
                    kit (SDK) that facilitates the optimization of AI and deep learning models for                 
                    deployment on these Neuron devices. It transforms models from popular                          
                    frameworks like PyTorch or TensorFlow into a specialized, low-level                            
                    representation for efficient execution on AWS Inferentia and Trainium.\n\nThe                  
                    AWS Neuron Kernel Mode Driver establishes a communication channel (FWIO) that                  
                    allows the driver and an application to send specific commands to the device.                  
                    \n\nAdditionally, there's a Neuron backend for Hugging Face's Text Generation                  
                    Inference (TGI) that allows deployment on AWS Trainium and Inferentia family                   
                    of chips.\n\nFor more detailed technical information, you can refer to the                     
                    provided links in the web search results.",                                                    
                            "refusal": null,                                                                       
                            "annotations": null,                                                                   
                            "audio": null,                                                                         
                            "function_call": null,                                                                 
                            "tool_calls": [],                                                                      
                            "reasoning_content": null                                                              
                          },                                           

AWS Neuron devices refer to the accelerated machine learning chips, specifically AWS Inferentia and AWS Trainium, which are designed to enhance AI and deep learning workloads on Amazon EC2 instances. These chips are optimized for high-density, scalable, and cost-effective inference and training respectively. 

AWS Neuron is a software development kit (SDK) that facilitates the optimization of AI and deep learning models for deployment on these Neuron devices. It transforms models from popular frameworks like PyTorch or TensorFlow into a specialized, low-level representation for efficient execution on AWS Inferentia and Trainium.

The AWS Neuron Kernel Mode Driver establishes a communication channel (FWIO) that allows the driver and an application to send specific commands to the device. 

Additionally, there's a Neuron backend for Hugging Face's Text Generation Inference (TGI) that allows deployment on AWS Trainium and Inferentia family of chips.

For more detailed technical informatio

In [121]:
from pprint import pprint
pprint(response.message["content"][0]["text"])

('AWS Neuron devices refer to the accelerated machine learning chips, '
 'specifically AWS Inferentia and AWS Trainium, which are designed to enhance '
 'AI and deep learning workloads on Amazon EC2 instances. These chips are '
 'optimized for high-density, scalable, and cost-effective inference and '
 'training respectively. \n'
 '\n'
 'AWS Neuron is a software development kit (SDK) that facilitates the '
 'optimization of AI and deep learning models for deployment on these Neuron '
 'devices. It transforms models from popular frameworks like PyTorch or '
 'TensorFlow into a specialized, low-level representation for efficient '
 'execution on AWS Inferentia and Trainium.\n'
 '\n'
 'The AWS Neuron Kernel Mode Driver establishes a communication channel (FWIO) '
 'that allows the driver and an application to send specific commands to the '
 'device. \n'
 '\n'
 "Additionally, there's a Neuron backend for Hugging Face's Text Generation "
 'Inference (TGI) that allows deployment on AWS Trai

Here’s a concise breakdown of what the logs show:

- **Initial payload:**  
  The first block (`payload=...`) is the request sent to the SageMaker endpoint.  
  It includes the user message *("Explain what is a Neuron device")*, the tool schema,  
  and generation parameters (`max_tokens`, `temperature`, etc.).

- **Model triggers a tool:**  
  The first response contains a `tool_call`, meaning the model decided  
  to use the `web_search` tool.  
  It sends arguments like:  
  `{"query": "what is a Neuron device", "max_results": 1}`.

- **Tool execution:**  
  The notebook executes the `web_search` function locally.  
  It runs the DuckDuckGo/Bing query and returns the results to the agent.

- **Second payload:**  
  The agent sends a new request to the endpoint that includes:  
  - the original user message  
  - the tool call  
  - the tool result  
  This allows the model to produce a final answer with the retrieved information.

- **Final model response:**  
  The last block shows the assistant's answer generated after incorporating  
  the tool's output. (In this example, the model misinterprets *Neuron device*,  
  but the flow is correct.)

Overall, this demonstrates the complete agent loop:
1. Model decides to call a tool  
2. Tool runs  
3. Result is injected back into the model  
4. Model produces the final answer  

✨ **The entire iteration—model → tool → model—completed in under 10 seconds.**

### 3. Use my Agent with HF MCP server 🤗

The **Hugging Face MCP (Model Context Protocol) Server** is a service that lets your MCP-compatible AI assistant (for example in VS Code, Cursor, Zed, or Claude Desktop) **connect directly to the Hugging Face Hub**. 

Once connected, your assistant can:
- Search and explore Hub resources (models, datasets, Spaces, papers).
- Invoke community tools (Gradio apps on Spaces) as functions, returning results—metadata, links, context.  

It effectively turns the Hugging Face Hub into a tool-server your agent can use, enhancing its tool-calling and exploration capabilities.

Learn more about the MCP server [here](https://huggingface.co/docs/hub/en/hf-mcp-server).

***🔧 Configure the MCP client***


To use the Hugging Face MCP server with your agent, you must first configure it at:  
👉 **https://huggingface.co/settings/mcp**

On this page, you can generate an MCP-compatible access token and configure which built-in tools your agent is allowed to use.

Once enabled, your agent gains access to a rich set of Hub-powered tools, including:

- **Spaces Semantic Search** – find the best AI apps on the Hub using natural language.  
- **Model Search** – query models by task, library, license, or metadata.  
- **Documentation Semantic Search** – search across all HF documentation.  
- **Hub Repository Details** – retrieve structured metadata for models, datasets, and Spaces.  
- **Papers Semantic Search** – find AI research papers semantically.  
- **Dataset Search** – search datasets by author, tags, modality, etc.  
- **Run and Manage Jobs** – schedule and monitor jobs on Hugging Face infrastructure.  
- **Include README files** – allow agents to read repository READMEs in results.

This integration lets your agent use the Hugging Face Hub as a powerful knowledge and tooling backend.


![MCP Settings](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/sagemaker/mcp-settings.png)

In [92]:
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
#MCP URL
hf_mcp_url = "https://huggingface.co/mcp"
hf_token = os.environ.get("HF_TOKEN")

mcp_client = MCPClient(
    lambda: streamablehttp_client(
        url=hf_mcp_url,
        headers={"Authorization": f"Bearer {hf_token}"} if hf_token else {},
        timeout=120,
        sse_read_timeout=600,
    )
)

In [93]:
#Use the MCP client as a tool
agent = Agent(model=model, tools=[mcp_client])

[23:58:56] INFO     HTTP Request: POST https://huggingface.co/mcp "HTTP/1.1 200 OK"                 ]8;id=646832;file:///home/ec2-user/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=14911;file:///home/ec2-user/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

           INFO     Received session ID: 815a36e0-a75a-4d45-822d-c57835cf405e                ]8;id=666447;file:///home/ec2-user/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=963313;file:///home/ec2-user/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py#134\134]8;;\

           INFO     Negotiated protocol version: 2025-06-18                                  ]8;id=594762;file:///home/ec2-user/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=959587;file:///home/ec2-user/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py#146\146]8;;\

           INFO     HTTP Request: GET https://huggingface.co/mcp "HTTP/1.1 405 Method Not Allowed"  ]8;id=580646;file:///home/ec2-user/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=587726;file:///home/ec2-user/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

           INFO     HTTP Request: POST https://huggingface.co/mcp "HTTP/1.1 202 Accepted"           ]8;id=22510;file:///home/ec2-user/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=637198;file:///home/ec2-user/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

           INFO     HTTP Request: POST https://huggingface.co/mcp "HTTP/1.1 200 OK"                 ]8;id=904597;file:///home/ec2-user/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=911302;file:///home/ec2-user/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

We are now ready to use the MCP client as a tool in our agent. we will test it to:
- Search for a Space that can transcribe audio files
- Search for trending image generation models
- Search the doc to export a model to neuron with optimum

***🔉 Space search***

In [94]:
audio_space_search = agent("Search for a Space that can transcribe audio files")
pprint(audio_space_search.message["content"][0]["text"])

[00:03:11] INFO     payload=<{                                                                     ]8;id=976710;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=245782;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#272\272]8;;\
                      "messages": [                                                                                
                        {                                                                                          
                          "role": "user",                                                                          
                          "content": [                                                                             
                            {                                                                                      
                              "text": "Search for a Space that can transcribe audio files",                        
                              "type": "text"                                                                       
                            }                                                                                      
                          ]                                                                                        
                        }                                                                                          
                      ],                                                                                           
                      "tools": [                                                                                   
                        {                                                                                          
                          "type": "function",                                                                      
                          "function": {                                                                            
                            "name": "hf_whoami",                                                                   
                            "description": "Hugging Face tools are being used by authenticated                     
                    user 'florentgbelidji'",                                                                       
                            "parameters": {                                                                        
                              "type": "object",                                                                    
                              "properties": {},                                                                    
                              "additionalProperties": false,                                                       
                              "$schema": "http://json-schema.org/draft-07/schema#",                                
                              "required": []                                                                       
                            }                                                                                      
                          }                                                                                        
                        },                                                                                         
                        {                                                                                          
                          "type": "function",                                                                      
                          "function": {                                                                            
                            "name": "space_search",                                                                
                            "description": "Find Hugging Face Spaces u

[00:03:15] INFO     response=<{                                                                    ]8;id=382428;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=840160;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#438\438]8;;\
                      "id": "chatcmpl-931f8fb6bc1e4c04aa95357340a46189",                                           
                      "object": "chat.completion",                                                                 
                      "created": 1764115391,                                                                       
                      "model":                                                                                     
                    "florentgbelidji/granite-3.3-8b-instruct-neuron-bs-1-seq-16384-tp-8",                          
                      "choices": [                                                                                 
                        {                                                                                          
                          "index": 0,                                                                              
                          "message": {                                                                             
                            "role": "assistant",                                                                   
                            "content": null,                                                                       
                            "refusal": null,                                                                       
                            "annotations": null,                                                                   
                            "audio": null,                                                                         
                            "function_call": null,                                                                 
                            "tool_calls": [                                                                        
                              {                                                                                    
                                "id": "chatcmpl-tool-583e805d016f4ebe85955f9abb412ed2",                            
                                "type": "function",                                                                
                                "function": {                                                                      
                                  "name": "space_search",                                                          
                                  "arguments": "{\"query\": \"transcribe audio files\", \"limit\":                 
                    5}"                                                                                            
                                }                                                                                  
                              }                                                                                    
                            ],                                                                                     
                            "reasoning_content": null                                                              
                          },                                                                                       
                          "logprobs": null,                                                                        
                          "finish_reason": "tool_calls",                                                           
                          "stop_reason": null,                                                                     
                          "token_ids": null                           


Tool #1: space_search


[00:03:16] INFO     HTTP Request: POST https://huggingface.co/mcp "HTTP/1.1 200 OK"                 ]8;id=316240;file:///home/ec2-user/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=108835;file:///home/ec2-user/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

           INFO     payload=<{                                                                     ]8;id=649824;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=837499;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#272\272]8;;\
                      "messages": [                                                                                
                        {                                                                                          
                          "role": "user",                                                                          
                          "content": [                                                                             
                            {                                                                                      
                              "text": "Search for a Space that can transcribe audio files",                        
                              "type": "text"                                                                       
                            }                                                                                      
                          ]                                                                                        
                        },                                                                                         
                        {                                                                                          
                          "role": "assistant",                                                                     
                          "tool_calls": [                                                                          
                            {                                                                                      
                              "function": {                                                                        
                                "arguments": "{\"query\": \"transcribe audio files\", \"limit\":                   
                    5}",                                                                                           
                                "name": "space_search"                                                             
                              },                                                                                   
                              "id": "chatcmpl-tool-583e805d016f4ebe85955f9abb412ed2",                              
                              "type": "function"                                                                   
                            }                                                                                      
                          ]                                                                                        
                        },                                                                                         
                        {                                                                                          
                          "role": "tool",                                                                          
                          "tool_call_id": "chatcmpl-tool-583e805d016f4ebe85955f9abb412ed2",                        
                          "content": "# Space Search Results for the query 'transcribe audio                       
                    files' (Showing 5 of 72 results)\n\n| Space | Description | Author | ID |                      
                    Category |  Likes | Trending Score | Relevance                                                 
                    |\n|-------|-------------|--------|----|----------|--------|----------------|-                 
                    ----------|\n| \ud83d\udcc9 [Whisper](https://hf.c

[00:03:24] INFO     response=<{                                                                    ]8;id=108272;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=413165;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#438\438]8;;\
                      "id": "chatcmpl-451e01508220478487c3913d8e34f1f0",                                           
                      "object": "chat.completion",                                                                 
                      "created": 1764115396,                                                                       
                      "model":                                                                                     
                    "florentgbelidji/granite-3.3-8b-instruct-neuron-bs-1-seq-16384-tp-8",                          
                      "choices": [                                                                                 
                        {                                                                                          
                          "index": 0,                                                                              
                          "message": {                                                                             
                            "role": "assistant",                                                                   
                            "content": "Here are some Hugging Face Spaces that can transcribe                      
                    audio files:\n\n1. [Whisper](https://hf.co/spaces/openai/whisper) by openai:                   
                    Transcribe audio files or YouTube videos into text. (Likes: 2620, Trending                     
                    Score: 14)\n2. [Whisper                                                                        
                    Turbo](https://hf.co/spaces/hf-audio/whisper-large-v3-turbo) by hf-audio:                      
                    Transcribe audio or YouTube videos into text. (Likes: 991, Trending Score:                     
                    4)\n3. [Faster Whisper Webui with                                                              
                    translate](https://hf.co/spaces/avans06/whisper-webui-translate) by avans06:                   
                    Transcribe audio files into text. (Likes: 145, Trending Score: 1)\n4.                          
                    [Realistic Text To Speech                                                                      
                    Unlimited](https://hf.co/spaces/NihalGazi/Text-To-Speech-Unlimited) by                         
                    NihalGazi: Free Text-To-Speech generator with Emotion control (OpenAI).                        
                    (Likes: 1657, Trending Score: 34)\n5.                                                          
                    [YourMT3](https://hf.co/spaces/mimbres/YourMT3) by mimbres: Transcribe audio                   
                    to MIDI. (Likes: 69, Trending Score: 8)\n\nThese Spaces offer various                          
                    functionalities for audio transcription, from basic transcription to more                      
                    advanced features like emotion control in text-to-speech conversion. Please                    
                    visit the links for more information and to explore each Space's                               
                    capabilities.",                                                                                
                            "refusal": null,                                                                       
                            "annotations": null,                                                                   
                            "audio": null,                            

Here are some Hugging Face Spaces that can transcribe audio files:

1. [Whisper](https://hf.co/spaces/openai/whisper) by openai: Transcribe audio files or YouTube videos into text. (Likes: 2620, Trending Score: 14)
2. [Whisper Turbo](https://hf.co/spaces/hf-audio/whisper-large-v3-turbo) by hf-audio: Transcribe audio or YouTube videos into text. (Likes: 991, Trending Score: 4)
3. [Faster Whisper Webui with translate](https://hf.co/spaces/avans06/whisper-webui-translate) by avans06: Transcribe audio files into text. (Likes: 145, Trending Score: 1)
4. [Realistic Text To Speech Unlimited](https://hf.co/spaces/NihalGazi/Text-To-Speech-Unlimited) by NihalGazi: Free Text-To-Speech generator with Emotion control (OpenAI). (Likes: 1657, Trending Score: 34)
5. [YourMT3](https://hf.co/spaces/mimbres/YourMT3) by mimbres: Transcribe audio to MIDI. (Likes: 69, Trending Score: 8)

These Spaces offer various functionalities for audio transcription, from basic transcription to more advanced features li

***📈 Trending model search***

In [95]:
image_model_search = agent("Search for trending image generation models")
pprint(image_model_search.message["content"][0]["text"])

[00:05:29] INFO     payload=<{                                                                     ]8;id=4828;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=50874;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#272\272]8;;\
                      "messages": [                                                                                
                        {                                                                                          
                          "role": "user",                                                                          
                          "content": [                                                                             
                            {                                                                                      
                              "text": "Search for a Space that can transcribe audio files",                        
                              "type": "text"                                                                       
                            }                                                                                      
                          ]                                                                                        
                        },                                                                                         
                        {                                                                                          
                          "role": "assistant",                                                                     
                          "tool_calls": [                                                                          
                            {                                                                                      
                              "function": {                                                                        
                                "arguments": "{\"query\": \"transcribe audio files\", \"limit\":                   
                    5}",                                                                                           
                                "name": "space_search"                                                             
                              },                                                                                   
                              "id": "chatcmpl-tool-583e805d016f4ebe85955f9abb412ed2",                              
                              "type": "function"                                                                   
                            }                                                                                      
                          ]                                                                                        
                        },                                                                                         
                        {                                                                                          
                          "role": "tool",                                                                          
                          "tool_call_id": "chatcmpl-tool-583e805d016f4ebe85955f9abb412ed2",                        
                          "content": "# Space Search Results for the query 'transcribe audio                       
                    files' (Showing 5 of 72 results)\n\n| Space | Description | Author | ID |                      
                    Category |  Likes | Trending Score | Relevance                                                 
                    |\n|-------|-------------|--------|----|----------|--------|----------------|-                 
                    ----------|\n| \ud83d\udcc9 [Whisper](https://hf.co/s

[00:05:34] INFO     response=<{                                                                    ]8;id=642942;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=483127;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#438\438]8;;\
                      "id": "chatcmpl-5c94969a334148f6a8a1f97ef2779a40",                                           
                      "object": "chat.completion",                                                                 
                      "created": 1764115530,                                                                       
                      "model":                                                                                     
                    "florentgbelidji/granite-3.3-8b-instruct-neuron-bs-1-seq-16384-tp-8",                          
                      "choices": [                                                                                 
                        {                                                                                          
                          "index": 0,                                                                              
                          "message": {                                                                             
                            "role": "assistant",                                                                   
                            "content": null,                                                                       
                            "refusal": null,                                                                       
                            "annotations": null,                                                                   
                            "audio": null,                                                                         
                            "function_call": null,                                                                 
                            "tool_calls": [                                                                        
                              {                                                                                    
                                "id": "chatcmpl-tool-55e038aab4574dd79ce0696c351efda5",                            
                                "type": "function",                                                                
                                "function": {                                                                      
                                  "name": "model_search",                                                          
                                  "arguments": "{\"query\": \"image generation\", \"sort\":                        
                    \"trendingScore\", \"limit\": 10}"                                                             
                                }                                                                                  
                              }                                                                                    
                            ],                                                                                     
                            "reasoning_content": null                                                              
                          },                                                                                       
                          "logprobs": null,                                                                        
                          "finish_reason": "tool_calls",                                                           
                          "stop_reason": null,                                                                     
                          "token_ids": null                           


Tool #2: model_search


           INFO     HTTP Request: POST https://huggingface.co/mcp "HTTP/1.1 200 OK"                 ]8;id=31815;file:///home/ec2-user/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=535249;file:///home/ec2-user/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

           INFO     payload=<{                                                                     ]8;id=849489;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=651568;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#272\272]8;;\
                      "messages": [                                                                                
                        {                                                                                          
                          "role": "user",                                                                          
                          "content": [                                                                             
                            {                                                                                      
                              "text": "Search for a Space that can transcribe audio files",                        
                              "type": "text"                                                                       
                            }                                                                                      
                          ]                                                                                        
                        },                                                                                         
                        {                                                                                          
                          "role": "assistant",                                                                     
                          "tool_calls": [                                                                          
                            {                                                                                      
                              "function": {                                                                        
                                "arguments": "{\"query\": \"transcribe audio files\", \"limit\":                   
                    5}",                                                                                           
                                "name": "space_search"                                                             
                              },                                                                                   
                              "id": "chatcmpl-tool-583e805d016f4ebe85955f9abb412ed2",                              
                              "type": "function"                                                                   
                            }                                                                                      
                          ]                                                                                        
                        },                                                                                         
                        {                                                                                          
                          "role": "tool",                                                                          
                          "tool_call_id": "chatcmpl-tool-583e805d016f4ebe85955f9abb412ed2",                        
                          "content": "# Space Search Results for the query 'transcribe audio                       
                    files' (Showing 5 of 72 results)\n\n| Space | Description | Author | ID |                      
                    Category |  Likes | Trending Score | Relevance                                                 
                    |\n|-------|-------------|--------|----|----------|--------|----------------|-                 
                    ----------|\n| \ud83d\udcc9 [Whisper](https://hf.c

[00:05:46] INFO     response=<{                                                                    ]8;id=22425;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=503824;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#438\438]8;;\
                      "id": "chatcmpl-f76c98f8ad0e463dbf02976b0fb06f84",                                           
                      "object": "chat.completion",                                                                 
                      "created": 1764115534,                                                                       
                      "model":                                                                                     
                    "florentgbelidji/granite-3.3-8b-instruct-neuron-bs-1-seq-16384-tp-8",                          
                      "choices": [                                                                                 
                        {                                                                                          
                          "index": 0,                                                                              
                          "message": {                                                                             
                            "role": "assistant",                                                                   
                            "content": "Here are some trending image generation models on Hugging                  
                    Face:\n\n1. **rocca/image-generation-js**: A JavaScript implementation for                     
                    image generation using Hugging Face models. (Trending Score: High)\n   -                       
                    [Link](https://hf.co/rocca/image-generation-js)\n\n2.                                          
                    **keras-io/GauGAN-Image-generation**: A TensorFlow-Keras implementation of                     
                    GauGAN for image generation based on segmentation maps. (Trending Score:                       
                    Moderate)\n   - [Link](https://hf.co/keras-io/GauGAN-Image-generation)\n\n3.                   
                    **mindlink/product-image-generation**: Generates images of products based on                   
                    textual descriptions. (Trending Score: Moderate)\n   -                                         
                    [Link](https://hf.co/mindlink/product-image-generation)\n\n4.                                  
                    **gligen/gligen-generation-text-image-box**: Uses Diffusers for text-to-image                  
                    generation with a box constraint. (Trending Score: Moderate)\n   -                             
                    [Link](https://hf.co/gligen/gligen-generation-text-image-box)\n\n5.                            
                    **aakash007/imagegeneration**: A collection of models and utilities for image                  
                    generation tasks. (Trending Score: Moderate)\n   -                                             
                    [Link](https://hf.co/aakash007/imagegeneration)\n\n6.                                          
                    **benlehrburger/animal-image-generation**: Generates images of animals from                    
                    textual descriptions using Stable Diffusion. (Trending Score: Moderate)\n   -                  
                    [Link](https://hf.co/benlehrburger/animal-image-generation)\n\n7.                              
                    **Craap/image-generation**: A collection of image generation models and                        
                    utilities. (Trending Score: Moderate)\n   -                                                    
                    [Link](https://hf.co/Craap/image-generation)\n\n8. 

Here are some trending image generation models on Hugging Face:

1. **rocca/image-generation-js**: A JavaScript implementation for image generation using Hugging Face models. (Trending Score: High)
   - [Link](https://hf.co/rocca/image-generation-js)

2. **keras-io/GauGAN-Image-generation**: A TensorFlow-Keras implementation of GauGAN for image generation based on segmentation maps. (Trending Score: Moderate)
   - [Link](https://hf.co/keras-io/GauGAN-Image-generation)

3. **mindlink/product-image-generation**: Generates images of products based on textual descriptions. (Trending Score: Moderate)
   - [Link](https://hf.co/mindlink/product-image-generation)

4. **gligen/gligen-generation-text-image-box**: Uses Diffusers for text-to-image generation with a box constraint. (Trending Score: Moderate)
   - [Link](https://hf.co/gligen/gligen-generation-text-image-box)

5. **aakash007/imagegeneration**: A collection of models and utilities for image generation tasks. (Trending Score: Moderate)

***📚 Documentation search***

In [96]:
doc_search = agent("How to export a model to neuron with optimum")
pprint(doc_search.message["content"][0]["text"])

[00:06:54] INFO     payload=<{                                                                     ]8;id=726480;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=8149;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#272\272]8;;\
                      "messages": [                                                                                
                        {                                                                                          
                          "role": "user",                                                                          
                          "content": [                                                                             
                            {                                                                                      
                              "text": "Search for a Space that can transcribe audio files",                        
                              "type": "text"                                                                       
                            }                                                                                      
                          ]                                                                                        
                        },                                                                                         
                        {                                                                                          
                          "role": "assistant",                                                                     
                          "tool_calls": [                                                                          
                            {                                                                                      
                              "function": {                                                                        
                                "arguments": "{\"query\": \"transcribe audio files\", \"limit\":                   
                    5}",                                                                                           
                                "name": "space_search"                                                             
                              },                                                                                   
                              "id": "chatcmpl-tool-583e805d016f4ebe85955f9abb412ed2",                              
                              "type": "function"                                                                   
                            }                                                                                      
                          ]                                                                                        
                        },                                                                                         
                        {                                                                                          
                          "role": "tool",                                                                          
                          "tool_call_id": "chatcmpl-tool-583e805d016f4ebe85955f9abb412ed2",                        
                          "content": "# Space Search Results for the query 'transcribe audio                       
                    files' (Showing 5 of 72 results)\n\n| Space | Description | Author | ID |                      
                    Category |  Likes | Trending Score | Relevance                                                 
                    |\n|-------|-------------|--------|----|----------|--------|----------------|-                 
                    ----------|\n| \ud83d\udcc9 [Whisper](https://hf.co/

[00:07:05] INFO     response=<{                                                                    ]8;id=45408;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py\sagemaker.py]8;;\:]8;id=65841;file:///home/ec2-user/.venv/lib/python3.13/site-packages/strands/models/sagemaker.py#438\438]8;;\
                      "id": "chatcmpl-35a13bd49ce24453affa8956c517d535",                                           
                      "object": "chat.completion",                                                                 
                      "created": 1764115615,                                                                       
                      "model":                                                                                     
                    "florentgbelidji/granite-3.3-8b-instruct-neuron-bs-1-seq-16384-tp-8",                          
                      "choices": [                                                                                 
                        {                                                                                          
                          "index": 0,                                                                              
                          "message": {                                                                             
                            "role": "assistant",                                                                   
                            "content": "To export a Hugging Face model to Neuron using Optimum,                    
                    you'll need to follow these steps:\n\n1. **Install Optimum**: If you haven't                   
                    already, install Optimum using pip:\n\n   ```bash\n   pip install optimum\n                    
                    ```\n\n2. **Choose a Model**: Select a model from the Hugging Face Model Hub                   
                    that you want to export. For example, let's use `bert-base-uncased` for a text                 
                    classification task.\n\n3. **Load the Model**: Use the `transformers` library                  
                    to load your chosen model.\n\n   ```python\n   from transformers import                        
                    AutoModelForSequenceClassification, AutoTokenizer\n\n   model_name =                           
                    \"bert-base-uncased\"\n   model =                                                              
                    AutoModelForSequenceClassification.from_pretrained(model_name)\n   tokenizer =                 
                    AutoTokenizer.from_pretrained(model_name)\n   ```\n\n4. **Prepare for                          
                    Export**: Optimum provides a `neuron_quantized` exporter. First, you need to                   
                    quantize your model.\n\n   ```python\n   from optimum.neuron.quantization                      
                    import NeuronQuantizer\n\n   quantizer = NeuronQuantizer(model)\n                              
                    quantized_model = quantizer.fit(model, dataset=your_dataset)\n   ```\n\n                       
                    Replace `your_dataset` with a suitable dataset for fine-tuning or                              
                    evaluation.\n\n5. **Export the Model**: Use the `export` method provided by                    
                    Optimum to export the quantized model to Neuron format.\n\n   ```python\n                      
                    from optimum.neuron.export import export_neuron\n\n                                            
                    export_neuron(quantized_model, \"bert-base-uncased-neuron\")\n   ```\n\n                       
                    This will save the exported model in Neuron format in a directory named                        
                    `bert-base-uncased-neuron`.\n\n6. **Load on Neuron**

To export a Hugging Face model to Neuron using Optimum, you'll need to follow these steps:

1. **Install Optimum**: If you haven't already, install Optimum using pip:

   ```bash
   pip install optimum
   ```

2. **Choose a Model**: Select a model from the Hugging Face Model Hub that you want to export. For example, let's use `bert-base-uncased` for a text classification task.

3. **Load the Model**: Use the `transformers` library to load your chosen model.

   ```python
   from transformers import AutoModelForSequenceClassification, AutoTokenizer

   model_name = "bert-base-uncased"
   model = AutoModelForSequenceClassification.from_pretrained(model_name)
   tokenizer = AutoTokenizer.from_pretrained(model_name)
   ```

4. **Prepare for Export**: Optimum provides a `neuron_quantized` exporter. First, you need to quantize your model.

   ```python
   from optimum.neuron.quantization import NeuronQuantizer

   quantizer = NeuronQuantizer(model)
   quantized_model = quantizer.fit(model, d

### 🧩 Understanding the Agent Output

The long log you see comes from the agent’s **full reasoning loop**, not just the final answer.  
Here’s what’s happening:

- The agent keeps a **persistent conversation history**, so past queries and tool calls are always included.
- When you run `agent("…")`, the model decides whether it should call a tool (e.g., HF MCP search).
- If a tool is needed, the agent:
  1. emits a `tool_call`,
  2. executes the tool,
  3. appends the tool result back into the conversation,
  4. asks the model again to produce the final answer.
- The printed value (`doc_search.message["content"][0]["text"]`) only shows the **assistant’s final answer**, but the log shows the **full chain**:  
  user message → tool call → tool output → final completion.